# Model Training

In [11]:
import os
import pandas as pd
import numpy as np

from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

# Fill in your Azure storage account information here
account_name = ''

## Load the preprocessed data

In [2]:
data_filename = 'wasb://model@{}.blob.core.windows.net/trainingdata'.format(account_name)
df = sqlContext.read.parquet(data_filename)

## Partition the data

In [5]:
train, test = df.randomSplit([0.8, 0.2], seed=0)
train = train.sampleBy('label', fractions={0.0: 0.2, 1.0: 0.8}, seed=0)

## Train the model

In [6]:
trained_model = RandomForestClassifier(featuresCol='features', labelCol='label').fit(train)

Store the model:

In [7]:
model_filename = 'wasb://model@{}.blob.core.windows.net/model'.format(account_name)
trained_model.save(model_filename)

## Evaluate the model

In [12]:
mc_evaluator = MulticlassClassificationEvaluator(labelCol='label')

predictions = trained_model.transform(test)
precision = mc_evaluator.evaluate(predictions, {mc_evaluator.metricName: "weightedPrecision"})
recall = mc_evaluator.evaluate(predictions, {mc_evaluator.metricName: "weightedRecall"})
accuracy = mc_evaluator.evaluate(predictions, {mc_evaluator.metricName: "accuracy"})
print('Accuracy: {:0.3f}\nWeighted precision: {:0.3f}\nWeighted recall: {:0.3f}\n'.format(accuracy, precision, recall))

Accuracy: 0.883
Weighted precision: 0.836
Weighted recall: 0.883